```{contents}
```
## Simple RNN

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import ParameterGrid


In [2]:

# Load IMDB dataset
max_words = 10000     # vocabulary size
maxlen = 200          # sequence length
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_words)


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [3]:

# Pad sequences
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)


In [4]:

# Function to create RNN model
def create_rnn_model(units=64, dropout_rate=0.2, learning_rate=0.001):
    model = Sequential([
        Embedding(max_words, 128, input_length=maxlen),
        SimpleRNN(units, activation='tanh'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


In [5]:

# Define parameter grid
param_grid = {
    'units': [32, 64],
    'dropout_rate': [0.2, 0.4],
    'learning_rate': [0.001, 0.0005]
}

best_acc = 0
best_params = {}


In [6]:

# Early stopping callback
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=2,            # stop if val_loss doesn’t improve for 2 epochs
    restore_best_weights=True,
    verbose=1
)


In [7]:

# Hyperparameter tuning (use small subset for speed)
for params in ParameterGrid(param_grid):
    print(f"Testing parameters: {params}")
    model = create_rnn_model(**params)
    history = model.fit(
        X_train[:5000], y_train[:5000],
        epochs=10, batch_size=64, verbose=0,
        validation_split=0.2,
        callbacks=[early_stop]
    )
    val_acc = max(history.history['val_accuracy'])
    print(f"Validation Accuracy: {val_acc:.4f}")
    
    if val_acc > best_acc:
        best_acc = val_acc
        best_params = params

print("\nBest Hyperparameters:", best_params)
print("Best Validation Accuracy:", best_acc)


Testing parameters: {'dropout_rate': 0.2, 'learning_rate': 0.001, 'units': 32}


C:\Users\sangouda\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 5: early stopping
Restoring model weights from the end of the best epoch: 3.
Validation Accuracy: 0.7950
Testing parameters: {'dropout_rate': 0.2, 'learning_rate': 0.001, 'units': 64}
Epoch 5: early stopping
Restoring model weights from the end of the best epoch: 3.
Validation Accuracy: 0.6690
Testing parameters: {'dropout_rate': 0.2, 'learning_rate': 0.0005, 'units': 32}
Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 4.
Validation Accuracy: 0.8050
Testing parameters: {'dropout_rate': 0.2, 'learning_rate': 0.0005, 'units': 64}
Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 4.
Validation Accuracy: 0.6680
Testing parameters: {'dropout_rate': 0.4, 'learning_rate': 0.001, 'units': 32}
Epoch 7: early stopping
Restoring model weights from the end of the best epoch: 5.
Validation Accuracy: 0.7740
Testing parameters: {'dropout_rate': 0.4, 'learning_rate': 0.001, 'units': 64}
Epoch 4: early stopping
Restoring model weights 

In [8]:

# Train final model with early stopping
final_model = create_rnn_model(**best_params)
final_model.fit(
    X_train, y_train,
    epochs=10, batch_size=64, validation_split=0.2,
    callbacks=[early_stop], verbose=1
)


Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 23s 63ms/step - accuracy: 0.6387 - loss: 0.6202 - val_accuracy: 0.7964 - val_loss: 0.4561
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 19s 61ms/step - accuracy: 0.8636 - loss: 0.3430 - val_accuracy: 0.8346 - val_loss: 0.3826
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 20s 63ms/step - accuracy: 0.9178 - loss: 0.2267 - val_accuracy: 0.8418 - val_loss: 0.3707
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 24s 76ms/step - accuracy: 0.9613 - loss: 0.1266 - val_accuracy: 0.8522 - val_loss: 0.3890
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 21s 67ms/step - accuracy: 0.9831 - loss: 0.0658 - val_accuracy: 0.8566 - val_loss: 0.4649
Epoch 5: early stopping
Restoring model weights from the end of the best epoch: 3.


In [9]:

# Evaluate on test data
test_loss, test_acc = final_model.evaluate(X_test, y_test, verbose=0)
print(f"\nTest Accuracy: {test_acc:.4f}")



Test Accuracy: 0.8449
